In [1]:
import json
import csv
import spacy
from tqdm import tqdm

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Đường dẫn đến file caption COCO
caption_file = "E:/Download/annotations/captions_val2017.json"

# Đọc file JSON
with open(caption_file, "r") as f:
    coco_data = json.load(f)

annotations = coco_data["annotations"]

# Danh sách triplet thu được
triplets = []

# Hàm đơn giản hoá tách triplet từ câu 
def extract_triplet(text):
    doc = nlp(text)
    for sent in doc.sents:
        for token in sent:
            if token.pos_ == "VERB":
                subj = [w for w in token.lefts if w.dep_ in ["nsubj", "nsubjpass"]]
                obj = [w for w in token.rights if w.dep_ in ["dobj", "pobj"]]
                if subj and obj:
                    return (subj[0].text.lower(), token.lemma_.lower(), obj[0].text.lower())
    return None

# Lặp qua caption
for ann in tqdm(annotations):
    caption = ann["caption"]
    image_id = ann["image_id"]
    triplet = extract_triplet(caption)
    if triplet:
        subject, predicate, obj = triplet
        triplets.append((subject, predicate, obj, image_id))

# Ghi ra file CSV
with open("f_coco_triplets.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["subject", "predicate", "object", "image_id"])
    writer.writerows(triplets)

print(f"✅ Đã ghi {len(triplets)} triplet vào f_coco_triplets.csv")


100%|███████████████████████████████████████████████████████████████████████████| 25014/25014 [03:10<00:00, 131.15it/s]

✅ Đã ghi 3316 triplet vào f_coco_triplets.csv
